<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLMEVALUATOR_OPENAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install lm-eval -q

In [ ]:
!pip install colab-env --quiet
!pip install tiktoken -q
!pip install accelerate -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q

In [ ]:
import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]
print(openai.api_key)

In [ ]:
#harness_repo="public-lm-eval-harness"

!git clone https://github.com/EleutherAI/lm-evaluation-harness/
%cd /content/lm-evaluation-harness

!pip install -e .

In [ ]:
!pip install datasets@git+https://github.com/huggingface/datasets.git@66d6242 -q
!pip install tokenizers>=0.15.2 transformers>=4.38.2 sentencepiece>=0.2.0 -q

In [ ]:
!which lm_eval

https://platform.openai.com/docs/guides/text-generation

In [17]:
%mkdir -p /content/lm_eval_output
%rm -rf /content/lm_eval_output/*

In [10]:
from openai import OpenAI

system_content = "You are a travel agent. Be descriptive and helpful."
user_content = "Tell me about Montreal"


client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=0.7,
    max_tokens=1024,
    #echo=True
)

response = chat_completion.choices[0].message.content
print("Together response:\n", response)

#TypeError: Completions.create() got an unexpected keyword argument 'echo'

Together response:
 Montreal, nestled in the heart of the Canadian province of Quebec, is a city steeped in a rich history and culture. It's the second-largest city in Canada and known for its stunning blend of old-world charm meets modern elegance. Being a bilingual city, both English and French are widely spoken, offering a European flair that's quite unique in North America.

The city is divided into 19 large boroughs, and each has its own distinctive character. Downtown Montreal is the central business district, filled with skyscrapers, shopping centers, and museums. Old Montreal, the historic district, is where you can find the enchanting cobblestone streets, horse-drawn carriages, and historic sites like the Notre-Dame Basilica. The Plateau Mont-Royal is known for its colorful houses, hip cafes, and vibrant nightlife.

Montreal is a city of festivals, hosting numerous large-scale events throughout the year, such as the world-famous Just for Laughs comedy festival, the Montreal In

https://platform.openai.com/docs/api-reference/models

In [45]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

[Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'),
 Model(id='davinci-002', created=1692634301, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-4o-2024-05-13', created=1715368132, object='model', owned_by='system'),
 Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'),
 Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'),
 Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'),
 Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'),
 Model(i

In [50]:
modelid=modellist.data[4].id
modelid

'gpt-4o-2024-05-13'

/content/lm-evaluation-harness/lm_eval/models/openai_completions.py -- line=478

In [11]:
import random
def loglikelihood(self, requests, disable_tqdm: bool = False):
        res = []

        for _ in tqdm(requests, disable=disable_tqdm):
            res.append((-random.random(), False))

        return res

https://github.com/EleutherAI/lm-evaluation-harness/

In [47]:
import os
#modelid='gpt-4'
#modelid='gpt-3.5-turbo-instruct'

os.environ['model'] = modelid
#os.environ['task']='mmlu'

#os.environ['task']='lambada_openai,hellaswag,mmlu'
os.environ['task']='lambada_openai,openbookqa,arc_easy,winogrande,hellaswag,arc_challenge,piqa,boolq'
os.environ['shot']='0'
os.environ['batch_size']='1'

os.environ['tokenizer']= modelid
os.environ['add_bos_token']='True'

os.environ['log_samples']='True'
os.environ['random_seed']='0'
os.environ['trust_remote_code']='True'

In [ ]:
#!lm_eval --model hf --model_args pretrained=EleutherAI/gpt-j-6b,parallelize=True,load_in_4bit=True,peft=nomic-ai/gpt4all-j-lora --tasks openbookqa,arc_easy,winogrande,hellaswag,arc_challenge,piqa,boolq --device cuda:0

In [48]:
!echo $model
!echo $task
!echo $shot
!echo $batch_size
#print(os.environ)
print()

# Supported model names: anthropic, anthropic-chat, anthropic-chat-completions, dummy, gguf, ggml, hf-auto, hf, huggingface, mamba_ssm, nemo_lm, sparseml, deepsparse, neuronx, openai-completions, local-completions, openai-chat-completions, local-chat-completions, openvino, textsynth, vllm
# trust_remote_code=True,
!lm_eval --model openai-chat-completions --model_args model=${model},tokenizer=${tokenizer}   --tasks ${task}  --num_fewshot ${shot} --output_path /content/lm_eval_output/${model//\//_}_${task//,/_}-${shot}shot --batch_size ${batch_size} 2>&1 | tee /content/lm_eval_output/eval-${model//\//_}_${task//,/_}-${shot}shot.log
#!lm_eval --model openai-completions --verbosity DEBUG --model_args model=${model},tokenizer=${tokenizer},random_seed=${random_seed},log_samples=${log_samples},trust_remote_code=${trust_remote_code}   --tasks ${task}  --num_fewshot ${shot} --output_path /content/lm_eval_output/${model//\//_}_${task//,/_}-${shot}shot --batch_size ${batch_size} 2>&1 | tee /content/lm_eval_output/eval-${model//\//_}_${task//,/_}-${shot}shot.log

gpt-4o-2024-05-13
lambada_openai,openbookqa,arc_easy,winogrande,hellaswag,arc_challenge,piqa,boolq
0
1

2024-05-14 10:16:53.474159: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 10:16:53.474288: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 10:16:53.482861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 10:16:55.953154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-14:10:17:01,821 INFO     [__main__.py:254] Verbosity set to INFO
2024-05-14:10:17:09,185 INFO     [__main__.py:341] Selected Task